<a href="https://colab.research.google.com/github/rishavsen1/gtfs_filtering/blob/main/trips_vid_clever_time_GTFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip bustime_chattanooga_jan_2022.json.zip

Archive:  bustime_chattanooga_jan_2022.json.zip
  inflating: bustime_chattanooga_jan_2022.json  


In [2]:
import json
with open("bustime_chattanooga_jan_2022.json", "r") as f:
    data = json.load(f)

dated = []
for record in data:
    tmstmp = record["tmstmp"]
    if "20220111" in tmstmp:

        dated.append(record)



In [17]:
data =dated
dated = []
for record in data:
    trp = record["origtatripno"]
    if trp:
        dated.append(trp)

unique_trp = list(set(dated))
unique_trp.sort()


In [3]:
for val in dated:
  val['coords'] = val['geometry']['coordinates']

In [ ]:
gdf.to_csv('20220111_spd_lat_long.geojson')

In [4]:
from datetime import time
data =dated
data_gen = []
vid = []
prev_trip_id = None
for record in data:
    trip_id = record["origtatripno"]
    if prev_trip_id != trip_id:
        vid = record['vid']
        srvtmstmp = record['srvtmstmp']
        data_gen.append({"trip_id": trip_id, "vid":vid, "start_time":srvtmstmp})
        prev_trip_id = trip_id

# unique_trp = list(set(data_gen))
# unique_trp.sort()
# data_gen
# print(len(unique_trp))

## Clever vid, trip id and GTFS times

In [14]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.9 MB/s 
     |████████████████████████████████| 16.7 MB 325 kB/s 
     |████████████████████████████████| 6.3 MB 45.1 MB/s 


In [19]:
import pandas as pd
data_gen_df = pd.DataFrame(data_gen, columns=["trip_id", 'vid', 'start_time'])
# data_gen_df

In [27]:
df_mod = data_gen_df.drop_duplicates(subset=["trip_id", "vid"]).reset_index(inplace=False).dropna()
df_mod["trip_id"] = df_mod["trip_id"]+"020"
df_mod = df_mod.drop(['index'], axis=1)
# df_mod = df_mod[["trip_id", "vid", 'start_time']]
df_mod = df_mod[df_mod.start_time > '20220111 04:00']

In [15]:
import gtfs_functions as gtfs
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./2021-10-24.zip')

In [55]:
import time
def val_to_time(val):
  return (time.strftime('%H:%M:%S', time.gmtime(val)))

In [56]:
df1 = df_mod
for unique in unique_trp:
    temp = stop_times[stop_times.trip_id == unique+'020'].sort_values('arrival_time')
    start = temp.arrival_time.iloc[0]
    end = temp.arrival_time.iloc[-1]
    t2 = df_mod.loc[df_mod.trip_id == unique+'020']
    df1.loc[t2.index.tolist()[0], 'gtfs_start_time'] = val_to_time(start)
    df1.loc[t2.index.tolist()[0], 'gtfs_end_time'] = val_to_time(end)
    

In [61]:
df1.to_csv('trips_vid_n_time_GTFS_20220111.csv', index=False)

In [ ]:
df_mod.to_csv('trips_vid_n_time_20220111.csv', index=False)